## Literature

https://quantpy.com.au/stochastic-calculus/brownian-motion-for-financial-mathematics/


https://www.youtube.com/watch?v=sIKD1tQryHg

https://www.quantstart.com/articles/geometric-brownian-motion-simulation-with-python/

https://towardsdatascience.com/stochastic-processes-simulation-brownian-motion-the-basics-c1d71585d9f9


https://github.com/GariZabaleta/jpx-tokyo-stock-exchange-prediction/blob/master/jpx-prediction.ipynb


https://towardsdatascience.com/5-amazing-pandas-features-you-probably-dont-know-about-5533498aac88


**Calculating log values**

https://stackoverflow.com/questions/31742545/python-calculating-log-returns-of-a-time-series



### Import libraries

---



In [1]:
import math
import itertools
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
import random
import seaborn as sns

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import neighbors
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [3]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

### Generating Geometric Brownian motion

---

In [4]:
# gbm.py

import os
import random
import string
import numpy as np
import pandas as pd


class GeometricBrownianMotionAssetSimulator:
    """
    This callable class will generate a daily
    open-high-low-close-volume (OHLCV) based DataFrame to simulate
    asset pricing paths with Geometric Brownian Motion for pricing
    and a Pareto distribution for volume.

    It will output the results to a CSV with a randomly generated
    ticker smbol.

    For now the tool is hardcoded to generate business day daily
    data between two dates, inclusive.

    Note that the pricing and volume data is completely uncorrelated,
    which is not likely to be the case in real asset paths.

    Parameters
    ----------
    start_date : `str`
        The starting date in YYYY-MM-DD format.
    end_date : `str`
        The ending date in YYYY-MM-DD format.
    init_price : `float`
        The initial price of the asset.
    mu : `float`
        The mean 'drift' of the asset.
    sigma : `float`
        The 'volatility' of the asset.
    """

    def __init__(
        self,
        start_date,
        end_date,
        init_price,
        mu,
        sigma,
    ):
        self.start_date = start_date
        self.end_date = end_date
        self.init_price = init_price
        self.mu = mu
        self.sigma = sigma

    def _generate_random_symbol(self):
        """
        Generates a random ticker symbol string composed of
        uppercase ASCII characters to use in the CSV output filename.

        Returns
        -------
        `str`
            The random ticker string composed of uppercase letters.
        """
        return ''.join(
            random.choices(
                string.ascii_uppercase,
                k=4
            )
        )

    def _create_empty_frame(self,symbol):
        """
        Creates the empty Pandas DataFrame with a date column using
        business days between two dates. Each of the price/volume
        columns are set to zero.

        Returns
        -------
        `pd.DataFrame`
            The empty OHLCV DataFrame for subsequent population.
        """
        date_range = pd.date_range(
            self.start_date,
            self.end_date,
            freq='B'
        )

        zeros = pd.Series(np.zeros(len(date_range)))

        return pd.DataFrame(
            {
                'date': date_range,
                symbol: zeros
            }
        )[['date', symbol]]

    def _create_geometric_brownian_motion(self, data):
        """
        Calculates an asset price path using the analytical solution
        to the Geometric Brownian Motion stochastic differential
        equation (SDE).

        This divides the usual timestep by four so that the pricing
        series is four times as long, to account for the need to have
        an open, high, low and close price for each day. These prices
        are subsequently correctly bounded in a further method.

        Parameters
        ----------
        data : `pd.DataFrame`
            The DataFrame needed to calculate length of the time series.

        Returns
        -------
        `np.ndarray`
            The asset price path (four times as long to include OHLC).
        """
        n = len(data)
        T = n / 252.0  # Business days in a year
        dt = T / (4.0 * n)  # 4.0 is needed as four prices per day are required
        
        # Vectorised implementation of asset path generation
        # including four prices per day, used to create OHLC
        asset_path = np.exp(
            (self.mu - self.sigma**2 / 2) * dt +
            self.sigma * np.random.normal(0, np.sqrt(dt), size=(4 * n))
        )
        
        return self.init_price * asset_path.cumprod()

    def _append_path_to_data(self, data, path,symbol):
        """
        Correctly accounts for the max/min calculations required
        to generate a correct high and low price for a particular
        day's pricing.

        The open price takes every fourth value, while the close
        price takes every fourth value offset by 3 (last value in
        every block of four).

        The high and low prices are calculated by taking the max
        (resp. min) of all four prices within a day and then
        adjusting these values as necessary.

        This is all carried out in place so the frame is not returned
        via the method.

        Parameters
        ----------
        data : `pd.DataFrame`
            The price/volume DataFrame to modify in place.
        path : `np.ndarray`
            The original NumPy array of the asset price path.
        """
        data[symbol] = path[3::4]


    def __call__(self):
        """
        The entrypoint for generating the asset OHLCV frame. Firstly this
        generates a symbol and an empty frame. It then populates this
        frame with some simulated GBM data. The asset volume is then appended
        to this data and finally it is saved to disk as a CSV.
        """
        symbol = self._generate_random_symbol()
        data = self._create_empty_frame()
        path = self._create_geometric_brownian_motion(data)
        self._append_path_to_data(data, path)


def gen(random_seed, start_date, end_date, init_price, mu, sigma):
    random_seed = int(random_seed)
    init_price = float(init_price)
    mu = float(mu)
    sigma = float(sigma)

    # Need to seed both Python and NumPy separately
    random.seed(random_seed)
    np.random.seed(seed=random_seed)

    gbmas = GeometricBrownianMotionAssetSimulator(
        start_date,
        end_date,
        init_price,
        mu,
        sigma,
    )
    return gbmas
     

In [5]:
start_data = "2017-01-01"
end_data = "2022-12-31"

In [6]:
df = []
for i in range(0,10):
  stock = gen(i, start_data, end_data,100,0.1,0.2)

  symbol = stock._generate_random_symbol()        
  data = stock._create_empty_frame(symbol)
  path = stock._create_geometric_brownian_motion(data)
  stock._append_path_to_data(data, path,symbol)

  if i == 0:
    df = data.copy()
  else:
    df = pd.concat([df,data[symbol]], axis = 1)

df = df.set_index("date")

In [7]:
display(df)

,VTKG,DWTG,YYBC,GOJP,GCKE,QTUY,UVMG,IDQB,FZDS,MJDW
date,,,,,,,,,,
2017-01-02,103.482514,99.661567,99.423036,100.320863,100.188276,101.484616,99.865500,101.086409,98.685564,99.142575
2017-01-03,104.622631,99.409426,97.360879,99.507999,98.932934,101.639198,98.671030,99.511884,101.265104,97.398578
2017-01-04,105.919790,99.110562,97.931147,98.945510,98.787116,100.689973,101.503570,100.059747,102.907880,98.346670
2017-01-05,107.067359,98.722860,97.265252,100.381085,98.788299,99.385565,102.268960,99.180151,101.040579,100.417669
2017-01-06,107.607129,98.490010,97.552069,99.024389,99.889633,100.767928,103.185455,98.853801,101.298598,101.596601
...,...,...,...,...,...,...,...,...,...,...
2022-12-26,99.701424,245.716194,56.282724,35.977166,241.754310,164.937524,379.217676,70.115651,169.591079,204.092912
2022-12-27,98.936285,243.107525,56.532859,35.316523,243.989987,161.603292,372.661570,69.507694,170.003221,210.684006
2022-12-28,99.408384,243.622433,55.611715,34.476697,240.949618,165.958196,371.707215,68.676143,168.345177,209.985817


In [8]:
n = random.randint(0,10)

X_train = df.iloc[:-5,:].copy()
X_train.drop(X_train.columns[n], axis=1, inplace = True)
y_train = df.iloc[:-5,n].copy()

X_test = df.iloc[-5:,:].copy()
X_test.drop(X_test.columns[n], axis=1, inplace = True)
y_test = df.iloc[-5:,n].copy()


 **Train**

In [9]:
print("---X_train----")
display(X_train)
print("\n--y_train----")
display(y_train)

---X_train----


,DWTG,YYBC,GOJP,GCKE,QTUY,UVMG,IDQB,FZDS,MJDW
date,,,,,,,,,
2017-01-02,99.661567,99.423036,100.320863,100.188276,101.484616,99.865500,101.086409,98.685564,99.142575
2017-01-03,99.409426,97.360879,99.507999,98.932934,101.639198,98.671030,99.511884,101.265104,97.398578
2017-01-04,99.110562,97.931147,98.945510,98.787116,100.689973,101.503570,100.059747,102.907880,98.346670
2017-01-05,98.722860,97.265252,100.381085,98.788299,99.385565,102.268960,99.180151,101.040579,100.417669
2017-01-06,98.490010,97.552069,99.024389,99.889633,100.767928,103.185455,98.853801,101.298598,101.596601
...,...,...,...,...,...,...,...,...,...
2022-12-19,240.982723,56.132685,35.041399,242.974425,164.449277,366.019465,72.452885,166.485526,205.637195
2022-12-20,241.853681,56.363396,36.062196,244.085363,165.776735,370.203239,72.934123,167.838902,205.536991
2022-12-21,249.873543,56.011898,36.329600,246.031651,162.518466,375.634517,73.751441,168.260879,208.878235



--y_train----


date
2017-01-02    103.482514
2017-01-03    104.622631
2017-01-04    105.919790
2017-01-05    107.067359
2017-01-06    107.607129
                 ...    
2022-12-19     94.167220
2022-12-20     96.616771
2022-12-21     97.161388
2022-12-22     99.606958
2022-12-23     97.767455
Name: VTKG, Length: 1560, dtype: float64

 **Test**

In [10]:
print("---X_test----")
display(X_test)
print("\n--y_test----")
display(y_test)

---X_test----


,DWTG,YYBC,GOJP,GCKE,QTUY,UVMG,IDQB,FZDS,MJDW
date,,,,,,,,,
2022-12-26,245.716194,56.282724,35.977166,241.754310,164.937524,379.217676,70.115651,169.591079,204.092912
2022-12-27,243.107525,56.532859,35.316523,243.989987,161.603292,372.661570,69.507694,170.003221,210.684006
2022-12-28,243.622433,55.611715,34.476697,240.949618,165.958196,371.707215,68.676143,168.345177,209.985817
2022-12-29,249.434677,56.437332,34.323396,236.256201,167.930676,369.841570,69.336655,167.931019,209.408823
2022-12-30,251.120725,57.051570,33.837276,234.204527,167.748045,375.111610,71.176126,167.914707,208.593867



--y_test----


date
2022-12-26     99.701424
2022-12-27     98.936285
2022-12-28     99.408384
2022-12-29    101.092194
2022-12-30    100.203742
Name: VTKG, dtype: float64

In [11]:
corr_mat = np.abs(X_train.corr())
fig, ax = plt.subplots(figsize=(10,5))
sns.heatmap(corr_mat,square = True, ax = ax)

NameError: ignored

In [ ]:
period = [3,5,10,21]
col = X_train.columns
for j in col:
  for i in period:
    X_train[f"pct_{j}_{i}"] = X_train[j].pct_change(i)
    X_train[f"pct_{j}_{i}"] = X_train[f"pct_{j}_{i}"].interpolate(method='linear',limit_direction='backward')

    X_train[f"Volatility_{j}_{i}"] = np.log(X_train[j]).diff().rolling(i).std()
    X_train[f"Volatility_{j}_{i}"] = X_train[f"Volatility_{j}_{i}"].interpolate(method='linear',limit_direction='backward')

In [ ]:
X_train

In [ ]:
X_train.drop(col, axis = 1, inplace = True)

In [ ]:
X_train

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.histplot(data=X_train["pct_IDQB_10"].values, bins=100,
             ax=ax)
ax.axvline(x=X_train["pct_IDQB_10"].mean(), color='red', linestyle='dotted', linewidth=2, 
           label='Mean')

ax.set_title("Target Mean Distibution Date\n"
             f"""Min {round(X_train["pct_IDQB_10"].min(), 4)} | """
             f"""Max {round(X_train["pct_IDQB_10"].max(), 4)} | """
             f"""Skewness {round(X_train["pct_IDQB_10"].skew(), 2)} | """
             f"""Kurtosis {round(X_train["pct_IDQB_10"].kurtosis(), 2)}""")

ax.set_xlabel("Target Mean")
ax.set_ylabel("Date Count")
ax.legend()
plt.show()


## Model selection

- K-Nearest Neighbour Regressor
- Decision tree
- Random Forest


In [ ]:
mae_dict = {"KNN":[],"Decision_Tree":[],"Random_Forest":[]}

**K-Nearest Neighbour Regressor**





In [ ]:
ts_fold = TimeSeriesSplit(n_splits=5,gap=10)
mae_ls = []

In [ ]:
%%time
for fold, (train_idx, val_idx) in enumerate(ts_fold.split(X_train, y_train)):
    
    print(f"\n========================== Fold {fold+1} ==========================")
        
    X_train_cross, y_train_cross = X_train.iloc[train_idx,:], y_train[train_idx]
    X_valid, y_val = X_train.iloc[val_idx,:], y_train[val_idx]
    
    print("Train Date range: {} to {}".format(X_train_cross.index.min(),X_train_cross.index.max()))
    print("Valid Date range: {} to {}".format(X_valid.index.min(),X_valid.index.max()))

    n_neighbors = 3
    knn_reg = neighbors.KNeighborsRegressor(n_neighbors)
    knn_reg.fit(X_train_cross, y_train_cross)
    y_pred = knn_reg.predict(X_valid)

    rmse = np.sqrt(mean_squared_error(y_train_cross, y_pred))
    mae = mean_absolute_error(y_train_cross, y_pred)

    print(f"\nMAE: {round(mae,2)}")
    mae_ls.append(mae)


print(f"\nMean MAE {round(sum(mae_ls)/len(mae_ls),2)}")
mae_dict["KNN"].append(round(sum(mae_ls)/len(mae_ls),2))

**Decision Tree** 



In [ ]:
ts_fold = TimeSeriesSplit(n_splits=5,gap=10)
mae_ls = []

In [ ]:
%%time
for fold, (train_idx, val_idx) in enumerate(ts_fold.split(X_train, y_train)):
    
    print(f"\n========================== Fold {fold+1} ==========================")
        
    X_train_cross, y_train_cross = X_train.iloc[train_idx,:], y_train[train_idx]
    X_valid, y_val = X_train.iloc[val_idx,:], y_train[val_idx]
    
    print("Train Date range: {} to {}".format(X_train_cross.index.min(),X_train_cross.index.max()))
    print("Valid Date range: {} to {}".format(X_valid.index.min(),X_valid.index.max()))

    tree_reg = DecisionTreeRegressor(random_state=42)
    tree_reg.fit(X_train_cross, y_train_cross)
    y_pred = tree_reg.predict(X_valid)

    rmse = np.sqrt(mean_squared_error(y_train_cross, y_pred))
    mae = mean_absolute_error(y_train_cross, y_pred)

    print(f"\nMAE: {round(mae,2)}")
    mae_ls.append(mae)


print(f"\nMean MAE {round(sum(mae_ls)/len(mae_ls),2)}")
mae_dict["KNN"].append(round(sum(mae_ls)/len(mae_ls),2))

**Random Forest**

In [ ]:
ts_fold = TimeSeriesSplit(n_splits=5,gap=10)
feat_importance_forest = pd.DataFrame()
mae_ls = []

In [ ]:
%%time
for fold, (train_idx, val_idx) in enumerate(ts_fold.split(X_train, y_train)):
    
    print(f"\n========================== Fold {fold+1} ==========================")
        
    X_train_cross, y_train_cross = X_train.iloc[train_idx,:], y_train[train_idx]
    X_valid, y_val = X_train.iloc[val_idx,:], y_train[val_idx]
    
    print("Train Date range: {} to {}".format(X_train_cross.index.min(),X_train_cross.index.max()))
    print("Valid Date range: {} to {}".format(X_valid.index.min(),X_valid.index.max()))

    regr = RandomForestRegressor()
    regr.fitX_train_cross, y_train_cross)
    y_pred = regr.predict(X_valid)

    rmse = np.sqrt(mean_squared_error(y_train_cross, y_pred))
    mae = mean_absolute_error(y_train_cross, y_pred)

    print(f"\nMAE: {round(mae,2)}")
    mae_ls.append(mae)


print(f"\nMean MAE {round(sum(mae_ls)/len(mae_ls),2)}")
mae_dict["KNN"].append(round(sum(mae_ls)/len(mae_ls),2))

**Selecting the best model**

In [ ]:
print(mae_dict)

Since Random Forest model provides a better MAE, it will be used to build the final model

In [ ]:
gari = hello +1

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
ts_fold = TimeSeriesSplit(n_splits=5,gap=10)
regr_final = RandomForestRegressor()

gsearch = GridSearchCV(estimator=regr_final, cv=ts_fold,
                       scoring = 'neg_mean_absolute_percentage_error',
                      param_grid=random_grid)
gsearch.fit(X, y)
results = gsearch.cv_results_

In [ ]:
ts_fold = TimeSeriesSplit(n_splits=5,gap=10)
regr_final = RandomForestRegressor()

gsearch = GridSearchCV(estimator=regr_final, cv=ts_fold,
                       scoring = 'neg_mean_absolute_percentage_error',
                      param_grid={'max_features': range(2, 50, 2)})
gsearch.fit(X, y)
results = gsearch.cv_results_

In [ ]:
results

In [ ]:
#plot the results
plt.figure(figsize=(13, 13))
plt.title("GridSearchCV",
          fontsize=16)

plt.xlabel("max_features")
plt.ylabel("Score")

ax = plt.gca()
ax.set_xlim(0, 50)


# Get the regular numpy array from the MaskedArray
X_axis = np.array(results['param_max_features'].data, dtype=float)


for sample, style in ('test', '-')
   sample_score_mean = (results['mean_%s_score' % (sample)])

In [ ]:
#plot the results
plt.figure(figsize=(13, 13))
plt.title("GridSearchCV",
          fontsize=16)

plt.xlabel("max_features")
plt.ylabel("Score")

ax = plt.gca()
ax.set_xlim(0, 50)


# Get the regular numpy array from the MaskedArray
X_axis = np.array(results['param_max_features'].data, dtype=float)


for sample, style in (('train', '--'), ('test', '-')):
    sample_score_mean = (results['mean_%s_score' % (sample)])
    sample_score_std = (results['std_%s_score' % (sample)])
    ax.fill_between(X_axis, sample_score_mean - sample_score_std,
                    sample_score_mean + sample_score_std,
                    alpha=0.1 if sample == 'test' else 0)
    ax.plot(X_axis, sample_score_mean, style,
            alpha=1 if sample == 'test' else 0.7,
            label="(%s)" % ( sample))

best_index = np.nonzero(results['rank_test_score' ] == 1)[0][0]
best_score =  (-results['mean_test_score' ][best_index])

# Plot a dotted vertical line at the best score for that scorer marked by x
ax.plot([X_axis[best_index], ] * 2, [best_score, best_score],
        linestyle='-.',  marker='x', markeredgewidth=3, ms=8)

# Annotate the best score for that scorer
ax.annotate("%0.2f" % best_score,
            (X_axis[best_index], best_score + 0.005))

plt.legend(loc="best")
plt.grid(False)
plt.show()

In [ ]:
feat_importance_forest['avg'] = feat_importance_forest.mean(axis=1)
feat_importance_forest = feat_importance_forest.sort_values(by='avg',ascending=True)
feat_importance_forest = feat_importance_forest.tail(20)
pal=sns.color_palette("plasma_r", 29).as_hex()[2:]

fig=go.Figure()
for i in range(len(feat_importance_forest.index)):
    fig.add_shape(dict(type="line", y0=i, y1=i, x0=0, x1=feat_importance_forest['avg'][i], 
                       line_color=pal[::-1][i],opacity=0.7,line_width=4))
    fig.add_trace(go.Scatter(x=feat_importance_forest['avg'], y=feat_importance_forest.index, mode='markers', 
                             marker_color=pal[::-1], marker_size=8,
                             hovertemplate='%{y} Importance = %{x:.0f}<extra></extra>'))
    fig.update_layout(title='Overall Feature Importance Random Forest', 
                      xaxis=dict(title='Average Importance',zeroline=False),
                      yaxis_showgrid=False, margin=dict(l=120,t=80),
                      height=700, width=800)
fig.show()